In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
# import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

# from tensorflow import keras
import keras
from keras.models import Sequential, load_model
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.layers.core import Reshape
from keras.layers.core import Dense, Activation, Dropout
# from keras.layers.cudnn_recurrent import CuDNNLSTM
from sklearn import preprocessing
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import backend as K

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
yscaler = StandardScaler()

Using TensorFlow backend.


In [2]:
close_df = pd.read_csv('../res/close_df.csv',index_col=0)
close_df.head()

df = np.log(close_df).diff()
target = df['XAU'].iloc[2:].values
features = df.iloc[1:-1].values

train_val_split = 6000
train_test_split = 7500

X_train = scaler.fit_transform(features[:train_val_split])
y_train = target[:train_val_split].reshape(-1,1)

X_val = scaler.transform(features[train_val_split:train_test_split])
y_val = target[train_val_split:train_test_split].reshape(-1,1)

X_test = scaler.transform(features[train_test_split:])
y_test = target[train_test_split:].reshape(-1,1)

def return_loss(y_true, y_pred):
    # pos = K.sign(y_pred)
    loss = -K.mean(y_pred * y_true)
    return loss

model = load_model('dnn_32_16.h5',custom_objects={'return_loss':return_loss})

In [11]:
# On the test set
test_df = pd.DataFrame()
test_df['y_true'] = y_test.flatten()
test_df['y_pred'] = model.predict(X_test).flatten()
test_df['label_true'] = test_df['y_true'].apply(lambda x: 1 if x > 0 else 0)
test_df['label_pred'] = test_df['y_pred'].apply(lambda x: 1 if x > 0 else 0)
test_df['correct'] = 1
test_df['correct'][test_df['label_true']!=test_df['label_pred']] = 0

# Part 2 区域性失效：给定窗口，收益不及预期
window = 50
step = 20
test_df['exp_rt'] = test_df['y_true'].rolling(window).sum() 
test_df['exp_rt'][test_df['exp_rt']<0] = 0
test_df['real_rt'] = (test_df['correct'].apply(lambda x: x if x == 1 else -1) * test_df['y_true']).rolling(window).sum()
test_df['fail'] = test_df[['exp_rt','real_rt']].apply(lambda x: 1 if x['real_rt']<x['exp_rt'] else 0,axis=1)

In [9]:
# Assume that we obtain testing criteria here
cri_df = pd.DataFrame(np.random.random((len(test_df),5)),columns=['NC','KMN','TKNC','LSA','DSA'])
cri_df.head()

,NC,KMN,TKNC,LSA,DSA
0,0.209581,0.786078,0.895980,0.058724,0.370563
1,0.361987,0.559075,0.613292,0.656974,0.321143
2,0.484138,0.065022,0.100501,0.460190,0.159362
3,0.306133,0.898114,0.733539,0.159541,0.176551
4,0.670294,0.975508,0.376054,0.567480,0.060587


In [12]:
test_df = pd.concat([test_df,cri_df],axis=1)
test_df.head()

,y_true,y_pred,label_true,label_pred,correct,exp_rt,real_rt,fail,NC,KMN,TKNC,LSA,DSA
0,0.013006,-0.995699,1,0,0,NaN,NaN,0,0.209581,0.786078,0.895980,0.058724,0.370563
1,0.004298,0.999448,1,1,1,NaN,NaN,0,0.361987,0.559075,0.613292,0.656974,0.321143
2,-0.008092,0.990847,0,1,0,NaN,NaN,0,0.484138,0.065022,0.100501,0.460190,0.159362
3,-0.006732,-0.962627,0,0,1,NaN,NaN,0,0.306133,0.898114,0.733539,0.159541,0.176551
4,-0.006475,-0.999999,0,0,1,NaN,NaN,0,0.670294,0.975508,0.376054,0.567480,0.060587


In [18]:
from statsmodels.discrete.discrete_model import Logit
from statsmodels.api import add_constant

In [19]:
data = test_df[['NC','KMN','TKNC','LSA','DSA']]
data = add_constant(data)
logitmodel = Logit(test_df['fail'],data)
result = logitmodel.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.668161
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   fail   No. Observations:                  737
Model:                          Logit   Df Residuals:                      731
Method:                           MLE   Df Model:                            5
Date:                Mon, 07 Dec 2020   Pseudo R-squ.:                0.008962
Time:                        10:30:33   Log-Likelihood:                -492.43
converged:                       True   LL-Null:                       -496.89
Covariance Type:            nonrobust   LLR p-value:                    0.1129
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8213      0.302     -2.719      0.007      -1.413      -0.229
NC            -0.0346      0.274     -0.126      0.900      -0.573       0.503
KMN            0.2558      0.259      0.989      0.322      -0.251       0.763
TKNC           0.2031      0.261      0.779      0.436      -0.308       0.714
LSA            0.6749      0.263      2.564      0.010       0.159       1.191
DSA           -0.2415      0.263     -0.917      0.359      -0.758       0.275
==============================================================================
"""